In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
data = []
labels = []

# Clean 폴더 이미지 불러오기
clean_dir = 'Data_PlasticCup/clean'
for filename in os.listdir(clean_dir):
    img = cv2.imread(os.path.join(clean_dir, filename))
    img = cv2.resize(img, (128, 128))
    data.append(img)
    labels.append(0)  # 0 = 깨끗한 컵

# NG 폴더 이미지 불러오기
ng_dir = 'Data_PlasticCup/ng'
for filename in os.listdir(ng_dir):
    img = cv2.imread(os.path.join(ng_dir, filename))
    img = cv2.resize(img, (128, 128))
    data.append(img)
    labels.append(1)  # 1 = 더러운 컵

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

In [3]:
# 모델 정의
model = Sequential()

# 컨볼루션 레이어 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 2 
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 컨볼루션 레이어 3
model.add(Conv2D(128, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))

# fully connected 레이어
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 데이터를 NumPy 배열로 변환
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=30, batch_size=16)

# 모델 저장
model.save('plastic_cup_classifier.h5')




Epoch 1/30


12/12 [==============================] - 2s 65ms/step - loss: 34.1854 - accuracy: 0.5243 - val_loss: 0.5904 - val_accuracy: 0.6809
Epoch 2/30
12/12 [==============================] - 1s 48ms/step - loss: 0.5765 - accuracy: 0.7568 - val_loss: 0.0691 - val_accuracy: 1.0000
Epoch 3/30
12/12 [==============================] - 1s 47ms/step - loss: 0.0914 - accuracy: 0.9676 - val_loss: 0.0045 - val_accuracy: 1.0000
Epoch 4/30
12/12 [==============================] - 1s 45ms/step - loss: 0.0085 - accuracy: 0.9946 - val_loss: 3.1805e-04 - val_accuracy: 1.0000
Epoch 5/30
12/12 [==============================] - 1s 45ms/step - loss: 0.0599 - accuracy: 0.9892 - val_loss: 0.0137 - val_accuracy: 1.0000
Epoch 6/30
12/12 [==============================] - 1s 45ms/step - loss: 0.0250 - accuracy: 0.9946 - val_loss: 0.0526 - val_accuracy: 1.0000
Epoch 7/30
12/12 [==============================] - 1s 45ms/step - loss: 0.2061 - accuracy: 0.9568 - val_loss: 0.0072 - val_accuracy: 1.0000
Epo

c:\Users\lavie\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model('plastic_cup_classifier.h5')

# 테스트 데이터셋 준비
x_test = np.array(x_val)
y_test = np.array(y_val)

# 모델 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc * 100:.2f}%')

# 예측값 구하기
y_pred = (model.predict(x_test) > 0.5).astype(int)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy (using sklearn.metrics): {accuracy * 100:.2f}%')

2/2 - 0s - loss: 9.3659e-10 - accuracy: 1.0000 - 116ms/epoch - 58ms/step
Test accuracy: 100.00%
2/2 [==============================] - 0s 10ms/step
Accuracy (using sklearn.metrics): 100.00%
